In [1]:
import pyomo.environ as pyo
import numpy as np
import json
import pandas as pd
from omlt import OmltBlock, OffsetScaling
from omlt.neuralnet import ReluComplementarityFormulation, ReluPartitionFormulation
from omlt.io.keras import keras_reader
from tensorflow import keras
from omlt.io.onnx import write_onnx_model_with_bounds, load_onnx_neural_network_with_bounds
import os
from sklearn.model_selection import train_test_split
from datetime import datetime
import contextlib

In [48]:
df = pd.read_csv('dataframe')
df = df[['FEED_BUT','FEED_PEN','FEED_HEX',"#-TRAYS","FEED_TRAY",'DIST_RATE','Reflux Rate','DIST_BUT','DIST_PEN','DIST_HEX','BOTT_BUT','BOTT_PEN','BOTT_HEX',"Investment Cost","Operating Cost"]]


feed_A = 330 #kmol/hr
feed_B = 330 #kmol/hr
feed_C = 340 #kmol/hr
feedrate = feed_A+feed_B+feed_C
def __init__():
    _ = 1
    return _
def formulation(sep_perc=.95,key='A'): #sep_perc is the purity percentage of the key as a decimal, key tells what compound will be seperated out
    dist_rate_range = (.25,.75) #25% of column input stream to 75%
    mol_A = feed_A/feedrate
    mol_B = feed_B/feedrate
    mol_C = feed_C/feedrate
    model = pyo.ConcreteModel()

    #sets
    model.COMP = pyo.Set(initialize=['A','B','C']) #A is n-butene, B is n-pentene, C is n-hexane
    model.COLUMNS = pyo.Set(initialize=['A|BC','AB|C'])
    model.STREAMS = pyo.Set(initialize=['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11',
                                   'F12','F13','F14','F15','F16','F17','F18','F19','F20','F21',
                                   'F22','F23']) #when referencing superstructure picture, subtract all streams by 1 so feed is 0 now and the last stream is 23
    model.PROD = pyo.Set(initialize=['P1','P2']) #products

    model.si = pyo.RangeSet(0,6) #index set to access splitters 2-8 in constraints
    model.ci = pyo.RangeSet(1,2) #index set to access columns 1-2
    model.fi = pyo.RangeSet(1,3) #index set to access streams connected to feed
    model.mci = pyo.RangeSet(0,1) #index set to access mixers 1 and 2
    model.mcp = pyo.RangeSet(0,1) #index set to access mixers 3 and 4
    #model.str_i = pyo.Set(initialize=[i for i in range(1, 23) if i not in [4,9]])
    model.str_i = pyo.RangeSet(1,23)
    inlet_split = [6,10,5,8,12,11,3] #indexes of streams that go into splitter to be called by model.f[i]
    outlet_split = [[1,2,3],[8,7],[13,12],[14,15],[16,17],[18,19],[20,21],[22,23]] #indexes of streams that go out of the splitter. First index corresponds to feed stream, not any of the streams in inlet_split list.
    inlet_column = [4,9] #inlet streams going into columns
    outlet_column = [[5,6],[10,11]]
    inlet_col_mix = [[1,13],[2,7]] #streams going into mixers before the column
    outlet_col_mix = [4,9] #streams going out of mixers before column
    inlet_prod_mix = [[14,16,18,20,22],[15,17,19,21,23]] #streams going out of mixers 3 and 4 going into products 1 and 2
    #data

    #Parameters

    #variables
    model.fa = pyo.VarList()
    for i in range(0,23):
        model.fa.add()
    for i in range(1, len(model.fa)+1):
        if i == 4 or i == 9:
            model.fa[i].setlb(0)
            model.fa[i].setub(330) #can raise upperbound because of recycle stream
        else:
            model.fa[i].setlb(0)
            model.fa[i].setub(500)
    model.fb = pyo.VarList()
    for i in range(0,23):
        model.fb.add()
    for i in range(1, len(model.fb)+1):
        if i == 4 or i == 9:
            model.fb[i].setlb(0)
            model.fb[i].setub(500) #can raise upperbound because of recycle stream
        else:
            model.fb[i].setlb(0)
            model.fb[i].setub(330)

    model.fc = pyo.VarList()
    for i in range(0,23):
        model.fc.add()
    for i in range(1, len(model.fc)+1):
        if i == 4 or i == 9:
            model.fc[i].setlb(0)
            model.fc[i].setub(340) #can raise upperbound because of recycle stream
        else:
            model.fc[i].setlb(0)
            model.fc[i].setub(340)

    model.ref_rate = pyo.VarList(domain=pyo.PositiveReals) #reflux rate
    model.dist_rate = pyo.VarList(initialize=0,bounds=(0,1000)) #distillate rate variable 0 - 1000
    model.num_trays = pyo.VarList(domain=pyo.NonNegativeIntegers,bounds=(3,22))
    model.feed_tray = pyo.VarList(domain=pyo.NonNegativeIntegers,bounds=(2,11))
    model.cap_cost = pyo.VarList()#domain=pyo.PositiveReals) 
    model.op_cost = pyo.VarList()#domain=pyo.PositiveReals)

    for i in range(1,len(model.COLUMNS)+1):
        model.ref_rate.add()
        model.dist_rate.add()
        model.num_trays.add()
        model.feed_tray.add()
        model.cap_cost.add()
        model.op_cost.add()

    #binary variables   
    model.y1 = pyo.Var(domain=pyo.Binary)
    model.y2 = pyo.Var(domain=pyo.Binary)

    #constraints
    #NN input variable constraints
    def ref_rate_constr1(model,i):
        return model.ref_rate[i] >= .75*model.dist_rate[i]

    def ref_rate_constr2(model,i):
        return model.ref_rate[i] <= 2*model.dist_rate[i]

    
    def dist_rate_constr1(model,i):
        ineq = model.dist_rate[i] >= dist_rate_range[0]*(model.fa[inlet_column[i-1]] + model.fb[inlet_column[i-1]] + model.fc[inlet_column[i-1]])
        return ineq
    def dist_rate_constr2(model,i):
        ineq = model.dist_rate[i] <= dist_rate_range[1]*(model.fa[inlet_column[i-1]] + model.fb[inlet_column[i-1]] + model.fc[inlet_column[i-1]])
        return ineq

    def feed_tray_constr(model,i):
        ineq = model.feed_tray[i] <= model.num_trays[i]/2
        return ineq

    #mass flow constraints
    #doing feed split constraints seperately because feed is constant so it's a slightly different form than the other split constraints
    def feed_splitA(model): #mass balance for first splitter
        ineq = feed_A == sum(model.fa[j] for j in outlet_split[0])
        return ineq
    def feed_mfA(model,i):
        ineq = mol_A*(model.fa[outlet_split[0][i-1]] + model.fb[outlet_split[0][i-1]] + model.fc[outlet_split[0][i-1]]) == model.fa[outlet_split[0][i-1]]
        return ineq

    def feed_splitB(model): #mass balance for first splitter
        ineq =  feed_B == sum(model.fb[j] for j in outlet_split[0])
        return ineq
    def feed_mfB(model,i):
        ineq = mol_B*(model.fa[outlet_split[0][i-1]] + model.fb[outlet_split[0][i-1]] + model.fc[outlet_split[0][i-1]]) == model.fb[outlet_split[0][i-1]]
        return ineq

    def feed_splitC(model): #making sure mole fracs are the same across streams 1 2 and 3
        ineq = feed_C == sum(model.fc[j] for j in outlet_split[0])
        return ineq
    def feed_mfC(model,i):
        ineq = mol_C*(model.fa[outlet_split[0][i-1]] + model.fb[outlet_split[0][i-1]] + model.fc[outlet_split[0][i-1]]) == model.fc[outlet_split[0][i-1]]
        return ineq

    #split constraints are making sure components flow in out streams add up to input component flow, and making sure mole ratios in out streams are same as mole ratios in in stream
    def split_flowA(model,i):
        ineq = model.fa[inlet_split[i]]== sum(model.fa[j] for j in outlet_split[i+1])
        return ineq
    def split_mfA(model,i,j):
        ineq = (model.fa[outlet_split[i+1][j-1]] +model.fb[outlet_split[i+1][j-1]] +model.fc[outlet_split[i+1][j-1]]) * model.fa[inlet_split[i]] \
        == model.fa[outlet_split[i+1][j-1]] * (model.fa[inlet_split[i]] + model.fb[inlet_split[i]] + model.fc[inlet_split[i]])
        return ineq

    def split_flowB(model,i,j):
        ineq = model.fb[inlet_split[i]]== sum(model.fb[j] for j in outlet_split[i+1])
        return ineq
    def split_mfB(model,i,j):
        ineq = (model.fa[outlet_split[i+1][j-1]] +model.fb[outlet_split[i+1][j-1]] +model.fc[outlet_split[i+1][j-1]]) * model.fb[inlet_split[i]] \
        == model.fb[outlet_split[i+1][j-1]] * (model.fa[inlet_split[i]] + model.fb[inlet_split[i]] + model.fc[inlet_split[i]])
        return ineq

    def split_flowC(model,i,j):
        ineq = model.fc[inlet_split[i]]== sum(model.fc[j] for j in outlet_split[i+1])
        return ineq
    def split_mfC(model,i,j):
        ineq = (model.fa[outlet_split[i+1][j-1]] +model.fb[outlet_split[i+1][j-1]] +model.fc[outlet_split[i+1][j-1]]) * model.fc[inlet_split[i]] \
        == model.fc[outlet_split[i+1][j-1]] * (model.fa[inlet_split[i]] + model.fb[inlet_split[i]] + model.fc[inlet_split[i]])
        return ineq

    #mixer constraints are making sure that component flow rates of in = out and making sure the amount mixed into the product streams is what we want.
    def mix_A_col(model,i):
        ineq = sum(model.fa[j]for j in inlet_col_mix[i]) == model.fa[outlet_col_mix[i]]
        #return (-.1,sum(model.f[j]*model.xa[j] for j in inlet_col_mix[i]) - model.f[outlet_col_mix[i]]*model.xa[outlet_col_mix[i]],.1)
        return ineq
    def mix_B_col(model,i):
        ineq = sum(model.fb[j] for j in inlet_col_mix[i]) == model.fb[outlet_col_mix[i]]
        #return (-.1,sum(model.f[j]*model.xb[j] for j in inlet_col_mix[i]) - model.f[outlet_col_mix[i]]*model.xb[outlet_col_mix[i]],.1)
        return ineq
    def mix_C_col(model,i):
        ineq = sum(model.fc[j]for j in inlet_col_mix[i]) == model.fc[outlet_col_mix[i]]
        #return (-.1,sum(model.f[j]*model.xc[j] for j in inlet_col_mix[i]) - model.f[outlet_col_mix[i]]*model.xc[outlet_col_mix[i]],.1)
        return ineq

    if key == 'A':
        def mix_key_prod(model):
            #ineq = sum(model.fa[j] for j in inlet_prod_mix[i]) == prod_comp['P1','A'] exact bounds
            return (sep_perc*feed_A,sum(model.fa[j] for j in inlet_prod_mix[0]),feed_A) #flow of A is either at seperation purity or above
        def mix_key_waste(model):
            #ineq = sum(model.f[j]*model.xa[j] for j in inlet_prod_mix[i]) == prod_comp['P2','A']
            return sum(model.fa[j] for j in inlet_prod_mix[1]) == feed_A - sum(model.fa[j] for j in inlet_prod_mix[0]) #flow of A in waste/recycle stream is opposite of product flow of A
        def mix_other_prod(model):
            #ineq = sum(model.fb[j] for j in inlet_prod_mix[i]) == prod_comp['P1','B']
            return (0,sum(model.fb[j] for j in inlet_prod_mix[0])+sum(model.fc[j] for j in inlet_prod_mix[0]),(1-sep_perc)*feed_A) #the flow rate of B and C is less than 1-seperation purity in the product stream
        def mix_other_waste1(model):
            #ineq = sum(model.fb[j] for j in inlet_prod_mix[i]) == prod_comp['P2','B']
            return sum(model.fb[j] for j in inlet_prod_mix[1]) == feed_B - sum(model.fb[j] for j in inlet_prod_mix[0])
        def mix_other_waste2(model):
            return sum(model.fc[j] for j in inlet_prod_mix[1]) == feed_C - sum(model.fc[j] for j in inlet_prod_mix[0])

    if key == 'B':
        print('doing b')
        def mix_key_prod(model):
            #ineq = sum(model.fa[j] for j in inlet_prod_mix[i]) == prod_comp['P1','A'] exact bounds
            return (sep_perc*feed_B,sum(model.fb[j] for j in inlet_prod_mix[0]),feed_B) #flow of B is either at seperation purity or above
        def mix_key_waste(model):
            #ineq = sum(model.f[j]*model.xa[j] for j in inlet_prod_mix[i]) == prod_comp['P2','A']
            return sum(model.fb[j] for j in inlet_prod_mix[1]) == feed_B - sum(model.fb[j] for j in inlet_prod_mix[0]) #flow of B in waste/recycle stream is opposite of product flow of B
        def mix_other_prod(model):
            #ineq = sum(model.fb[j] for j in inlet_prod_mix[i]) == prod_comp['P1','B']
            return (0,sum(model.fa[j] for j in inlet_prod_mix[0])+sum(model.fc[j] for j in inlet_prod_mix[0]),(1-sep_perc)*feed_B) #the flow rate of B and C is less than 1-seperation purity in the product stream
        def mix_other_waste1(model):
            #ineq = sum(model.fb[j] for j in inlet_prod_mix[i]) == prod_comp['P2','B']
            return sum(model.fa[j] for j in inlet_prod_mix[1]) == feed_A - sum(model.fa[j] for j in inlet_prod_mix[0])
        def mix_other_waste2(model):
            return sum(model.fc[j] for j in inlet_prod_mix[1]) == feed_C - sum(model.fc[j] for j in inlet_prod_mix[0])

    if key == 'C':
        def mix_key_prod(model):
            #ineq = sum(model.fa[j] for j in inlet_prod_mix[i]) == prod_comp['P1','A'] exact bounds
            return (sep_perc*feed_C,sum(model.fc[j] for j in inlet_prod_mix[0]),feed_C) #flow of A is either at seperation purity or above
        def mix_key_waste(model):
            #ineq = sum(model.f[j]*model.xa[j] for j in inlet_prod_mix[i]) == prod_comp['P2','A']
            return sum(model.fc[j] for j in inlet_prod_mix[1]) == feed_C - sum(model.fc[j] for j in inlet_prod_mix[0]) #flow of A in waste/recycle stream is opposite of product flow of A
        def mix_other_prod(model):
            #ineq = sum(model.fb[j] for j in inlet_prod_mix[i]) == prod_comp['P1','B']
            return (0,sum(model.fb[j] for j in inlet_prod_mix[0])+sum(model.fa[j] for j in inlet_prod_mix[0]),(1-sep_perc)*feed_C) #the flow rate of B and C is less than 1-seperation purity in the product stream
        def mix_other_waste1(model):
            #ineq = sum(model.fb[j] for j in inlet_prod_mix[i]) == prod_comp['P2','B']
            return sum(model.fb[j] for j in inlet_prod_mix[1]) == feed_B - sum(model.fb[j] for j in inlet_prod_mix[0])
        def mix_other_waste2(model):
            return sum(model.fa[j] for j in inlet_prod_mix[1]) == feed_A - sum(model.fa[j] for j in inlet_prod_mix[0])

    #column constraints
    scale = np.array(df.std(ddof=0))
    mean = np.array(df.mean())
    print(df)
    meanx = mean[:7]
    meany = mean[7:]
    xscale = scale[:7]
    yscale = scale[7:]
    
    
    print('xmean is ')
    print(meanx)
    print('ymean is ')
    print(meany)
    print('xscale is ')
    print(xscale)
    print('yscale is ')
    print(yscale)
    A = np.array([[1, 0, 0, 0, 0, 0, 0],
                                    [0, 1, 0, 0, 0, 0, 0],
                                  [0 , 0, 1, 0, 0, 0, 0]])

    B = np.array([[-1, 0, 0, -1, 0, 0, 0, 0,],
                                    [0, -1, 0, 0, -1, 0, 0, 0],
                                  [0, 0, -1, 0, 0, -1, 0, 0]])

    b = np.array([0, 0,0])
    A_scaled = A * xscale
    B_scaled = B * yscale
    b_scaled = b

    chunk = B_scaled.T @ np.linalg.inv(B_scaled@B_scaled.T)

    Astar = - chunk @ A_scaled
    Bstar = np.identity(8) - chunk@B_scaled
    bstar = chunk@B_scaled
    print('A star is ')
    print(Astar)
    print('B star is')
    print(Bstar)
    model.nn_ab = OmltBlock()
    model.nn_bc = OmltBlock()

    net_relu_ab = load_onnx_neural_network_with_bounds('KKThPINN_dist 33.onnx')

    net_relu_bc = load_onnx_neural_network_with_bounds('KKThPINN_dist 33.onnx')

    formulation_ab = ReluPartitionFormulation(net_relu_ab)
    formulation_bc = ReluPartitionFormulation(net_relu_bc)

    model.nn_ab.build_formulation(formulation_ab)
    model.nn_bc.build_formulation(formulation_bc)
    
    
    @model.Constraint()
    #column 1 inputs
    def col1_fa(mdl):
        return mdl.fa[4]  == (mdl.nn_ab.inputs[0]*xscale[0]+meanx[0]) * mdl.y1
    @model.Constraint()
    def col1_fb(mdl):
        return mdl.fb[4] == (mdl.nn_ab.inputs[1]*xscale[1]+meanx[1]) * mdl.y1
    @model.Constraint()
    def col1_fc(mdl):
        return  mdl.fc[4] == (mdl.nn_ab.inputs[2]*xscale[2]+meanx[2]) * mdl.y1
    @model.Constraint()
    def col1_num_trays(mdl):
        return  mdl.num_trays[1]  == (mdl.nn_ab.inputs[3]*xscale[3]+meanx[3])
    @model.Constraint()
    def col1_feed_tray(mdl):
        return mdl.feed_tray[1] == (mdl.nn_ab.inputs[4] * xscale[4]+meanx[4])
    @model.Constraint()
    def col1_dist_rate(mdl):
        return mdl.dist_rate[1] == (mdl.nn_ab.inputs[5] * xscale[5] + meanx[5]) * mdl.y1
    @model.Constraint()
    def col1_ref_rate(mdl):
        return mdl.ref_rate[1] == (mdl.nn_ab.inputs[6] * xscale[6] + meanx[6]) * mdl.y1

    #column 2 inputs
    @model.Constraint()
    def col2_fa(mdl):
        return mdl.fa[9] ==  (mdl.nn_bc.inputs[0]*xscale[0]+meanx[0]) * mdl.y2
    @model.Constraint()
    def col2_fb(mdl):
        return mdl.fb[9] == (mdl.nn_bc.inputs[1]*xscale[1]+meanx[1]) * mdl.y2
    @model.Constraint()
    def col2_fc(mdl):
        return mdl.fc[9] == (mdl.nn_bc.inputs[2]*xscale[2]+meanx[2]) * mdl.y2
    @model.Constraint()
    def col2_num_trays(mdl):
        return  mdl.num_trays[2]  == (mdl.nn_bc.inputs[3]*xscale[3]+meanx[3])
    @model.Constraint()
    def col2_feed_tray(mdl):
        return mdl.feed_tray[2] == (mdl.nn_bc.inputs[4] * xscale[4]+meanx[4]) 
    @model.Constraint()
    def col2_dist_rate(mdl):
        return mdl.dist_rate[2] == (mdl.nn_bc.inputs[5] * xscale[5] + meanx[5]) * mdl.y2
    @model.Constraint()
    def col2_ref_rate(mdl):
        return mdl.ref_rate[2] == (mdl.nn_bc.inputs[6] * xscale[6] + meanx[6]) * mdl.y2

    #columns 1 outputs
    @model.Constraint()
    def col1_da(mdl):
        #return mdl.fa[5]==(mdl.nn_ab.inputs[0]*yscale[0]+meany[0])*mdl.y1
        return mdl.fa[5] == ((mdl.nn_ab.inputs[0]*Astar[0][0] + mdl.nn_ab.outputs[0]*Bstar[0][0] + 
                              mdl.nn_ab.outputs[3]*Bstar[0][3])*yscale[0] + meany[0]) * mdl.y1 #applying projection layers
    @model.Constraint()
    def col1_db(mdl):
        #return mdl.fb[5] == (mdl.nn_ab.outputs[1]*yscale[1]+meany[1])*mdl.y1
        return mdl.fb[5] == ((mdl.nn_ab.inputs[1]*Astar[1][1] + mdl.nn_ab.outputs[1]*Bstar[1][1] + 
                              mdl.nn_ab.outputs[4]*Bstar[1][4])*yscale[1] + meany[1]) * mdl.y1
    @model.Constraint()
    def col1_dc(mdl):
        #return mdl.fc[5] == (mdl.nn_ab.outputs[2]*yscale[2]+meany[2])*mdl.y1
        return mdl.fc[5] == ((mdl.nn_ab.inputs[2]*Astar[2][2] + mdl.nn_ab.outputs[2]*Bstar[2][2] + 
                              mdl.nn_ab.outputs[5]*Bstar[2][5])*yscale[2] + meany[2]) * mdl.y1

    @model.Constraint()
    def col1_ba(mdl):
        #return mdl.fa[6] == (mdl.nn_ab.outputs[3]*yscale[3]+meany[3])*mdl.y1
        return mdl.fa[6] == ((mdl.nn_ab.inputs[0]*Astar[3][0] + mdl.nn_ab.outputs[0]*Bstar[3][0] + 
                              mdl.nn_ab.outputs[3]*Bstar[3][3])*yscale[3] + meany[3]) * mdl.y1

    @model.Constraint()
    def col1_bb(mdl):
        #return mdl.fb[6] == (mdl.nn_ab.outputs[4]*yscale[4]+meany[4])*mdl.y1
        return mdl.fb[6] == ((mdl.nn_ab.inputs[1]*Astar[4][1] + mdl.nn_ab.outputs[1]*Bstar[4][1] + 
                              mdl.nn_ab.outputs[4]*Bstar[4][4])*yscale[4] + meany[4]) * mdl.y1
    @model.Constraint()
    def col1_bc(mdl):
        #return mdl.fc[6] = (mdl.nn_ab.outputs[5]*yscale[5]+meany[5])*mdl.y1
        return mdl.fc[6] == ((mdl.nn_ab.inputs[2]*Astar[5][2] + mdl.nn_ab.outputs[2]*Bstar[5][2] + 
                              mdl.nn_ab.outputs[5]*Bstar[5][5])*yscale[5] + meany[5]) * mdl.y1
    @model.Constraint()
    def col1_cap_cost(mdl):
        return mdl.cap_cost[1] == (mdl.nn_ab.outputs[6]*yscale[6]+meany[6]) * mdl.y1
    @model.Constraint()
    def col1_op_cost(mdl):
        return mdl.op_cost[1] == (mdl.nn_ab.outputs[7]*yscale[7]+meany[7]) * mdl.y1

    #column 2 outputs
    @model.Constraint()
    def col2_da(mdl):
        return mdl.fa[10] == ((mdl.nn_bc.inputs[0]*Astar[0][0] + mdl.nn_bc.outputs[0]*Bstar[0][0] + 
                              mdl.nn_bc.outputs[3]*Bstar[0][3])*yscale[0] + meany[0]) * mdl.y2 #applying projection layers
     
    @model.Constraint()
    def col2_db(mdl):
        return mdl.fb[10] == ((mdl.nn_bc.inputs[1]*Astar[1][1] + mdl.nn_bc.outputs[1]*Bstar[1][1] + 
                              mdl.nn_bc.outputs[4]*Bstar[1][4])*yscale[1] + meany[1]) * mdl.y2
        
    @model.Constraint()
    def col2_dc(mdl):
        return mdl.fc[10] == ((mdl.nn_bc.inputs[2]*Astar[2][2] + mdl.nn_bc.outputs[2]*Bstar[2][2] + 
                              mdl.nn_bc.outputs[5]*Bstar[2][5])*yscale[2] + meany[2]) * mdl.y2
        
    @model.Constraint()
    def col2_ba(mdl):
        return mdl.fa[11] == ((mdl.nn_bc.inputs[0]*Astar[3][0] + mdl.nn_bc.outputs[0]*Bstar[3][0] + 
                              mdl.nn_bc.outputs[3]*Bstar[3][3])*yscale[3] + meany[3]) * mdl.y2
        
    @model.Constraint()
    def col2_bb(mdl):
        return mdl.fb[11] == ((mdl.nn_bc.inputs[1]*Astar[4][1] + mdl.nn_bc.outputs[1]*Bstar[4][1] + 
                              mdl.nn_bc.outputs[4]*Bstar[4][4])*yscale[4] + meany[4]) * mdl.y2
        
    @model.Constraint()
    def col2_bc(mdl):
        return mdl.fc[11] == ((mdl.nn_bc.inputs[2]*Astar[5][2] + mdl.nn_bc.outputs[2]*Bstar[5][2] + 
                              mdl.nn_bc.outputs[5]*Bstar[5][5])*yscale[5] + meany[5]) * mdl.y2
    
    @model.Constraint()
    def col2_cap_cost(mdl):
        return mdl.cap_cost[2] == (mdl.nn_bc.outputs[6]*yscale[6]+meany[6])  * mdl.y2
    @model.Constraint()
    def col2_op_cost(mdl):
        return mdl.op_cost[2] == (mdl.nn_bc.outputs[7]*yscale[7]+meany[7]) * mdl.y2


    model.ref_rate_constr1 = pyo.Constraint(model.ci,rule=ref_rate_constr1)
    model.ref_rate_constr2 = pyo.Constraint(model.ci,rule=ref_rate_constr2)

    model.dist_rate_constr1 = pyo.Constraint(model.ci,rule=dist_rate_constr1)
    model.dist_rate_constr2 = pyo.Constraint(model.ci,rule=dist_rate_constr2)
    
    model.feed_tray_constr = pyo.Constraint(model.ci,rule=feed_tray_constr)

    model.feed_splitA_constr = pyo.Constraint(rule=feed_splitA)
    model.feed_mfA_constr = pyo.Constraint(model.fi,rule=feed_mfA)
    model.feed_splitB_constr = pyo.Constraint(rule=feed_splitB)
    model.feed_mfB_constr = pyo.Constraint(model.fi,rule=feed_mfB)
    model.feed_splitC_constr = pyo.Constraint(rule=feed_splitC)
    model.feed_mfC_constr = pyo.Constraint(model.fi,rule=feed_mfC)

    model.splitA_constr = pyo.Constraint(model.si,rule=split_flowA)
    model.split_mfA_constr = pyo.Constraint(model.si,model.ci,rule=split_mfA)
    model.splitB_constr = pyo.Constraint(model.si,model.ci,rule=split_flowB)
    model.split_mfB_constr = pyo.Constraint(model.si,model.ci,rule=split_mfB)
    model.splitC_constr = pyo.Constraint(model.si,model.ci,rule=split_flowC)
    model.split_mfC_constr = pyo.Constraint(model.si,model.ci,rule=split_mfC)

    model.mix_A_col_constr = pyo.Constraint(model.mci,rule=mix_A_col)
    model.mix_B_col_constr = pyo.Constraint(model.mci,rule=mix_B_col)
    model.mix_C_col_constr = pyo.Constraint(model.mci,rule=mix_C_col)


    model.mix_key_prod_constr = pyo.Constraint(rule=mix_key_prod)
    model.mix_key_waste_constr = pyo.Constraint(rule=mix_key_waste)
    model.mix_other_prod_constr = pyo.Constraint(rule=mix_other_prod)
    model.mix_other_waste1_constr = pyo.Constraint(rule=mix_other_waste1)
    model.mix_other_waste2_constr = pyo.Constraint(rule=mix_other_waste2)

    #Objective function
    model.obj = pyo.Objective(expr=(model.cap_cost[1] + model.cap_cost[2] + model.op_cost[1] + model.op_cost[2]),sense=pyo.minimize)
    return model

def validation_save(model):
    feed_a1 = pyo.value(model.fa[4])
    feed_b1 = pyo.value(model.fb[4])
    feed_c1 = pyo.value(model.fc[4])
    dist_a1 = pyo.value(model.fa[5])
    dist_b1 = pyo.value(model.fb[5])
    dist_c1 = pyo.value(model.fc[5])
    bott_a1 = pyo.value(model.fa[6])
    bott_b1 = pyo.value(model.fb[6])
    bott_c1 = pyo.value(model.fc[6])
    ref_rate1 = pyo.value(model.ref_rate[1])
    dist_rat1 = pyo.value(model.dist_rate[1])
    cap_cost1 = pyo.value(model.cap_cost[1])*1e4
    op_cost1 = pyo.value(model.op_cost[1])*1e4
    col1_vals = [feed_a1,feed_b1,feed_c1,dist_a1,dist_b1,dist_c1,bott_a1,bott_b1,bott_c1,ref_rate1,dist_rat1,cap_cost1,op_cost1]

    feed_a2 = pyo.value(model.fa[9])
    feed_b2 = pyo.value(model.fb[9])
    feed_c2 = pyo.value(model.fc[9])
    dist_a2 = pyo.value(model.fa[10])
    dist_b2 = pyo.value(model.fb[10])
    dist_c2 = pyo.value(model.fc[10])
    bott_a2 = pyo.value(model.fa[11])
    bott_b2 = pyo.value(model.fb[11])
    bott_c2 = pyo.value(model.fc[11])
    ref_rate2 = pyo.value(model.ref_rate[2])
    dist_rat2 = pyo.value(model.dist_rate[2])
    cap_cost2 = pyo.value(model.cap_cost[2])*1e4
    op_cost2 = pyo.value(model.op_cost[2])*1e4
    col2_vals = [feed_a2,feed_b2,feed_c2,dist_a2,dist_b2,dist_c2,bott_a2,bott_b2,bott_c2,ref_rate2,dist_rat2,cap_cost2,op_cost2]

    df_save = pd.DataFrame([col1_vals,col2_vals], columns=['FEED_BUT','FEED_PEN','FEED_HEX','DIST_BUT','DIST_PEN','DIST_HEX','BOTT_BUT',
                                                 'BOTT_PEN','BOTT_HEX','REF_RATE','DIST_RATE','CAP_COST','OP_COST'])
    df_save.to_csv('kkt_val_values/{}-{} validation values'.format(sep_perc,key))
                       
    return model        
                
def save_output_to_file(file,model,sep_perc,key):
    with open(file, 'w') as f:
        with contextlib.redirect_stdout(f):
            print("Product 1 A flowrate is {:.3f} ".format(pyo.value(model.fa[14]) + pyo.value(model.fa[16])
                                                                + pyo.value(model.fa[18]) + pyo.value(model.fa[20])
                                                                + pyo.value(model.fa[22])))

            print("Product 2 A flowrate is {:.3f}".format(pyo.value(model.fa[15]) + pyo.value(model.fa[17])
                                                                + pyo.value(model.fa[19]) + pyo.value(model.fa[21])
                                                                + pyo.value(model.fa[23])))

            print("Product 1 B flowrate is {:.3f}".format(pyo.value(model.fb[14]) + pyo.value(model.fb[16])
                                                                + pyo.value(model.fb[18]) + pyo.value(model.fb[20])
                                                                + pyo.value(model.fb[22])))

            print("Product 2 B flowrate is {:.3f} ".format(pyo.value(model.fb[15]) + pyo.value(model.fb[17])
                                                                + pyo.value(model.fb[19]) + pyo.value(model.fb[21])
                                                                + pyo.value(model.fb[23])))
            print("Product 1 C flowrate is {:.3f}".format(pyo.value(model.fc[14]) + pyo.value(model.fc[16])
                                                                + pyo.value(model.fc[18]) + pyo.value(model.fc[20])
                                                                + pyo.value(model.fc[22])))

            print("Product 2 C flowrate is {:.3f}".format(pyo.value(model.fc[15]) + pyo.value(model.fc[17])
                                                                + pyo.value(model.fc[19]) + pyo.value(model.fc[21])
                                                                + pyo.value(model.fc[23])))

            print("Column 1 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[4]) - pyo.value(model.fa[5])
                                                                                             - pyo.value(model.fa[6]), pyo.value(model.fb[4])
                                                                                             - pyo.value(model.fb[5]) - pyo.value(model.fb[6]),
                                                                                             pyo.value(model.fc[4]) - pyo.value(model.fc[5])
                                                                                             - pyo.value(model.fc[6])))
            print("Column 2 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[9]) - pyo.value(model.fa[10])
                                                                                             - pyo.value(model.fa[11]), pyo.value(model.fb[9])
                                                                                             - pyo.value(model.fb[10]) - pyo.value(model.fb[11]),
                                                                                             pyo.value(model.fc[9]) - pyo.value(model.fc[10])
                                                                                             - pyo.value(model.fc[11])))
            print("Mixer 1 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[4]) - pyo.value(model.fa[13])
                                                                                             - pyo.value(model.fa[1]), pyo.value(model.fb[4])
                                                                                             - pyo.value(model.fb[13]) - pyo.value(model.fb[1]),
                                                                                             pyo.value(model.fc[4]) - pyo.value(model.fc[13])
                                                                                             - pyo.value(model.fc[1])))
            print("Mixer 2 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[9]) - pyo.value(model.fa[7])
                                                                                             - pyo.value(model.fa[2]), pyo.value(model.fb[9])
                                                                                             - pyo.value(model.fb[7]) - pyo.value(model.fb[2]),
                                                                                             pyo.value(model.fc[9]) - pyo.value(model.fc[7])
                                                                                             - pyo.value(model.fc[2])))
            print("Splitter 1 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(feed_A - pyo.value(model.fa[1])
                                                                                             - pyo.value(model.fa[2]) - pyo.value(model.fa[3]), feed_B
                                                                                             - pyo.value(model.fb[1]) - pyo.value(model.fb[2]) - pyo.value(model.fb[3]),
                                                                                             feed_C - pyo.value(model.fc[1])
                                                                                             - pyo.value(model.fc[2]) - pyo.value(model.fc[3])))
            print("Splitter 2 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[6]) - pyo.value(model.fa[7])
                                                                                             - pyo.value(model.fa[8]), pyo.value(model.fb[6])
                                                                                             - pyo.value(model.fb[7]) - pyo.value(model.fb[8]),
                                                                                             pyo.value(model.fc[6]) - pyo.value(model.fc[7])
                                                                                             - pyo.value(model.fc[8])))
            print("Splitter 3 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[10]) - pyo.value(model.fa[13])
                                                                                             - pyo.value(model.fa[12]), pyo.value(model.fb[10])
                                                                                             - pyo.value(model.fb[13]) - pyo.value(model.fb[12]),
                                                                                             pyo.value(model.fc[10]) - pyo.value(model.fc[13])
                                                                                             - pyo.value(model.fc[12])))
            print("Splitter 4 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[5]) - pyo.value(model.fa[14])
                                                                                             - pyo.value(model.fa[15]), pyo.value(model.fb[5])
                                                                                             - pyo.value(model.fb[14]) - pyo.value(model.fb[15]),
                                                                                             pyo.value(model.fc[5]) - pyo.value(model.fc[14])
                                                                                             - pyo.value(model.fc[15])))
            print("Splitter 5 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[8]) - pyo.value(model.fa[16])
                                                                                             - pyo.value(model.fa[17]), pyo.value(model.fb[8])
                                                                                             - pyo.value(model.fb[16]) - pyo.value(model.fb[17]),
                                                                                             pyo.value(model.fc[8]) - pyo.value(model.fc[16])
                                                                                             - pyo.value(model.fc[17])))
            print("Splitter 6 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[12]) - pyo.value(model.fa[18])
                                                                                             - pyo.value(model.fa[19]), pyo.value(model.fb[12])
                                                                                             - pyo.value(model.fb[18]) - pyo.value(model.fb[19]),
                                                                                             pyo.value(model.fc[12]) - pyo.value(model.fc[18])
                                                                                             - pyo.value(model.fc[19])))
            print("Splitter 7 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[11]) - pyo.value(model.fa[20])
                                                                                             - pyo.value(model.fa[21]), pyo.value(model.fb[11])
                                                                                             - pyo.value(model.fb[20]) - pyo.value(model.fb[21]),
                                                                                             pyo.value(model.fc[11]) - pyo.value(model.fc[20])
                                                                                             - pyo.value(model.fc[21])))
            print("Splitter 8 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[3]) - pyo.value(model.fa[23])
                                                                                             - pyo.value(model.fa[22]), pyo.value(model.fb[3])
                                                                                             - pyo.value(model.fb[23]) - pyo.value(model.fb[22]),
                                                                                             pyo.value(model.fc[3]) - pyo.value(model.fc[23])
                                                                                             - pyo.value(model.fc[22])))
            print("\n \n")
            print("Col 1 capital cost is ${:.2f}".format(pyo.value(model.cap_cost[1])*1e4))
            print("Col 2 capital cost is ${:.2f}".format(pyo.value(model.cap_cost[2])*1e4))
            print("Col 1 operating cost is ${:.2f}".format(pyo.value(model.op_cost[1])*1e4))
            print("Col 2 operating cost is ${:.2f}".format(pyo.value(model.op_cost[2])*1e4))
            print("\n")
           
            print("Col 1 reflux rate is {:.3f}".format(pyo.value(model.ref_rate[1])))
            print("Col 1 dist rate is {:.3f}".format(pyo.value(model.dist_rate[1])))
            print("Col 1 # trays is {:.3f}".format(pyo.value(model.num_trays[1])))
            print("Col 1 feed tray is {:.3f}".format(pyo.value(model.feed_tray[1])))
            
            print("\n")
           
            print("Col 2 reflux rate is {:.3f}".format(pyo.value(model.ref_rate[2])))
            print("Col 2 dist rate is {:.3f}".format(pyo.value(model.dist_rate[2])))
            print("Col 2 # trays is {:.3f}".format(pyo.value(model.num_trays[2])))
            print("Col 2 feed tray is {:.3f}".format(pyo.value(model.feed_tray[2])))            
            print("\n")
            for i in range(1,24):
                print("Stream {} A flowrate is {:.3f}".format(i,pyo.value(model.fa[i])))
                print("Stream {} B flowrate is {:.3f}".format(i,pyo.value(model.fb[i])))
                print("Stream {} C flowrate is {:.3f}".format(i,pyo.value(model.fc[i])))

            prod_A = pyo.value(model.fa[14]) + pyo.value(model.fa[16]) + pyo.value(model.fa[18]) + pyo.value(model.fa[20]) + pyo.value(model.fa[22])
            prod_B = pyo.value(model.fb[14]) + pyo.value(model.fb[16])+ pyo.value(model.fb[18]) + pyo.value(model.fb[20])+ pyo.value(model.fb[22])
            prod_C = pyo.value(model.fc[14]) + pyo.value(model.fc[16]) + pyo.value(model.fc[18]) + pyo.value(model.fc[20])+ pyo.value(model.fc[22])
            print("Purity of A in product stream is {:.3f}".format(prod_A / (prod_A+prod_B+prod_C)))
            print("Purity of B in product stream is {:.3f}".format(prod_B / (prod_A+prod_B+prod_C)))
            print("Purity of C in product stream is {:.3f}".format(prod_C / (prod_A+prod_B+prod_C)))
                
    

In [64]:
save_file = True
sep_perc = .68
key = 'B'

if save_file == True:
    model = formulation(sep_perc=sep_perc,key=key)
    solver = pyo.SolverFactory('gurobi')
    solver.options["LogFile"] = 'kkt_log_files/32 NN SIMPLE {}-{}'.format(sep_perc,key)
    status = solver.solve(model, tee=True, options={'TimeLimit': 1200,'MIPFocus': 2})# options={'TimeLimit': 3600}
else:
    model = formulation(sep_perc=sep_perc,key=key)
    solver = pyo.SolverFactory('gurobi')
    status = solver.solve(model, tee=True,options={'TimeLimit': 45})# options={'TimeLimit': 3600}
validation_save(model)
save_output_to_file('kkt_output_values/32 NN  {}-{}'.format(sep_perc,key) + " output values",model,sep_perc,key)

doing b
        FEED_BUT    FEED_PEN    FEED_HEX  #-TRAYS  FEED_TRAY   DIST_RATE  \
0      24.769228    5.646050   23.041407       18          3   21.327593   
1     841.208060   17.572812   94.466675        9          9  510.265925   
2      27.580601   79.821017  254.607065       17          7  229.104042   
3     150.338592  135.413796  358.958613        6          5  194.325655   
4      64.647803  294.821212  262.953554       12          8  203.744185   
...          ...         ...         ...      ...        ...         ...   
6781  216.886176  113.643310  131.728285        6          4  337.906331   
6782  103.222135  389.462263   69.599602       17         10  184.568234   
6783  137.264095   15.452323    0.928725        9          3   76.230172   
6784  488.670596   18.070199  364.140738       18         16  490.421180   
6785   17.817305   53.678899   49.028757       10          3   81.839197   

      Reflux Rate    DIST_BUT    DIST_PEN    DIST_HEX    BOTT_BUT    BOTT_PEN  

ValueError: Cannot load a SolverResults object with bad status: aborted

In [62]:
model.pprint()

4 Set Declarations
    COLUMNS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'A|BC', 'AB|C'}
    COMP : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}
    PROD : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'P1', 'P2'}
    STREAMS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {'F0', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23'}

6 RangeSet Declarations
    ci : Dimen=1, Size=2, Bounds=(1, 2)
        Key  : Finite : Members
        None :   True :   [1:2]
    fi : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1: